1-2 задания

In [1]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import re


class Account:
    _account_counter = 1000

    def __init__(self, account_holder, balance=0.0):
        # корректность имени владельца
        if not re.match(r'^[A-ZА-Я][a-zа-я]+\s[A-ZА-Я][a-zа-я]+$', account_holder):
            raise ValueError("Имя владельца должно быть в формате 'Имя Фамилия' с заглавных букв")
        Account._account_counter += 1
        self.holder = account_holder
        self.account_number = f"ACC-{Account._account_counter}"
        self._balance = balance
        self.operations_history = []


    # Запись операций для истории
    def _record(self, op_type, amount, status):
        self.operations_history.append({
            "type": op_type,
            "amount": amount,
            "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "balance_after": self._balance,
            "status": status
        })


    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Сумма пополнения должна быть положительной")
        self._balance += amount
        self._record("deposit", amount, "success")


    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной")
        if amount > self._balance:
            self._record("withdraw", amount, "fail")
            raise ValueError("Недостаточно средств")
        self._balance -= amount
        self._record("withdraw", amount, "success")


    def get_balance(self):
        return self._balance
    

    def get_history(self):
        return self.operations_history

    def history_analytics(self, amount_threshold=100, last_n=5):
        filtered_by_amount = []
        for operation in self.operations_history:
            if operation['amount'] > amount_threshold:
                filtered_by_amount.append(operation)
        
        filtered_by_amount.sort(key=lambda x: x['time'], reverse=True)
        values_to_show = min(last_n, len(filtered_by_amount))
        result = filtered_by_amount[:values_to_show]

        return result


# Класс CheckingAccount (расчётный счёт)
class CheckingAccount(Account):
    account_type = "Checking"

    def __init__(self, account_holder, balance=0):
        super().__init__(account_holder, balance)


# Класс SavingsAccount (сберегательный счёт)
class SavingsAccount(Account):
    account_type = "Savings"

    def __init__(self, account_holder, balance=0):
        super().__init__(account_holder, balance)


    def withdraw(self, amount):
        if amount > self._balance * 0.5:
            self._record("withdraw", amount, "fail")
            raise ValueError("Нельзя снять больше 50% от текущего баланса")
        super().withdraw(amount)


    def apply_interest(self, rate):
        if rate <= 0:
            raise ValueError("Процентная ставка должна быть положительной")
        interest = self._balance * rate / 100
        self._balance += interest
        self._record("interest", interest, "success")
        print(f"Начислены проценты: {interest:.2f} ({rate}%)")

In [2]:
base_acc = Account("Илья Фомиченко", 228.0)
base_acc.deposit(100)
base_acc.deposit(1337)
base_acc.deposit(1000000)

In [3]:
base_acc.get_balance()

1001665.0

In [4]:
base_acc.history_analytics()

[{'type': 'deposit',
  'amount': 1337,
  'time': '2025-11-08 23:50:46',
  'balance_after': 1665.0,
  'status': 'success'},
 {'type': 'deposit',
  'amount': 1000000,
  'time': '2025-11-08 23:50:46',
  'balance_after': 1001665.0,
  'status': 'success'}]

In [5]:
saving_acc = SavingsAccount("Илья Фомиченко")
saving_acc.deposit(100)
saving_acc.withdraw(70)

ValueError: Нельзя снять больше 50% от текущего баланса

In [6]:
saving_acc_2 = SavingsAccount("Илья Фомиченко")
saving_acc_2.deposit(100)
saving_acc_2.withdraw(30)
saving_acc_2.apply_interest(50)
saving_acc_2.get_balance()

Начислены проценты: 35.00 (50%)


105.0